In [5]:
import os
import torch
import pickle
from wavegan import *
from utils import *
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
def save_samples(epoch_samples, epoch, output_dir, fs=16000):
    sample_dir = os.path.join(output_dir, str(epoch))
    if not os.path.exists(sample_dir):
        os.makedirs(sample_dir)

    for idx, samp in enumerate(epoch_samples):
        output_path = os.path.join(sample_dir, "{}.wav".format(idx+1))
        samp = samp[0]
        librosa.output.write_wav(output_path, samp, sr=fs)

In [7]:
output_path = 'output/test'
gen_path = 'output/generator.pkl'
data_path = 'dataset/lr'
window_length = 16384
model_size = 64
sample_size = 5
ngpus = 4
idx = 145

In [8]:
lr_data = GeneratorDataset(data_path, idx, window_length)
dataloader = DataLoader(lr_data, batch_size=10, collate_fn=lr_data.collate_fn)

dataset/lr/SI1303.WAV


In [9]:
model_gen = torch.nn.DataParallel(WaveGANGenerator(model_size=model_size, ngpus=ngpus)).to(device)
model_gen.load_state_dict(torch.load(gen_path))

/home/mega4alik/madina/WaveGAN-pytorch-master/wavegan.py:54: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


RuntimeError: CUDA out of memory. Tried to allocate 1024.00 MiB (GPU 0; 10.75 GiB total capacity; 1.13 GiB already allocated; 324.12 MiB free; 1.15 GiB reserved in total by PyTorch)

In [ ]:
for idx, lr_data in enumerate(dataloader):
    lr_data_Var = numpy_to_var(lr_data, device)
    sample_out = model_gen(lr_data_Var)
    print(sample_out)
    sample_out = sample_out.to(device)
    sample_out = sample_out.data.cpu().numpy()
    save_samples(sample_out, idx, output_path)
    
    lr_data_Var = lr_data_Var.data.cpu().numpy()
    save_samples(lr_data_Var, idx+1, output_path)

In [ ]:
audio_list = []
for r, d, f in os.walk(output_path):
    f.sort()
    for i, file in enumerate(f):
        audio, _ = librosa.load(os.path.join(output_path, '0', file), sr=16000)
        if i%2==0:
            audio = audio[2000:]
            audio_list.append(audio)
        else:
            audio = audio[:-2000]
            audio_list.append(audio)

In [ ]:
audio_output = np.concatenate(audio_list)

In [ ]:
audio_output.shape

In [ ]:
librosa.output.write_wav('output.wav', audio_output, sr=16000)

In [ ]:
lr_audio_files = get_all_audio_filepaths(data_path)
for idx, i in enumerate(lr_audio_files):
    if 'SI1303' in i:
        print(idx)

In [ ]:
a = lr_audio_files[145]
a, _ = librosa.load(a, sr=16000)
librosa.output.write_wav('a.wav', a, sr=16000)

In [10]:
lr_audio_files = get_all_audio_filepaths('dataset/hr')
for idx, i in enumerate(lr_audio_files):
    if 'SI1303' in i:
        print(idx)

145


In [12]:
a = lr_audio_files[145]
a, _ = librosa.load(a, sr=16000)
librosa.output.write_wav('hr_SI1303.wav', a, sr=16000)